# gpu ram drive

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  3 14:11:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import math
warnings.filterwarnings('ignore')

In [5]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 30 kB/s 


# dataset

In [5]:
def load_data(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) 
        train = pd.concat([train,data])
    print('finish!!')
    return train

In [8]:
train = load_data('drive/MyDrive/dacon/data_re/상추/train_input/*.csv')

finish!!


In [9]:
train.describe()

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
count,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000,18816.000000
mean,13.500000,11.500000,3.266345,4.146259,6.189912,0.660002,4.544143,7.847381,4.034263,6.428896,2.919746,4.830358,2.527593,4.190245,5.925521,9.017202
std,8.077962,6.922371,0.157941,0.208276,0.200678,0.461405,2.840180,1.904948,4.684538,5.727701,3.868949,4.974876,3.719785,4.872833,4.557179,4.745604
min,0.000000,0.000000,1.945910,2.302585,4.110874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.750000,5.750000,3.178054,4.060443,6.042633,0.000000,0.000000,7.118826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.893058
50%,13.500000,11.500000,3.295837,4.143135,6.161207,0.693147,5.533389,8.117014,0.000000,10.052166,0.000000,0.000000,0.000000,0.000000,9.131622,11.433211
75%,20.250000,17.250000,3.367296,4.304065,6.313548,1.098612,6.646391,9.208138,9.811208,11.887402,7.310550,9.771726,7.175490,9.831508,9.812249,12.086789
max,27.000000,23.000000,3.713572,4.543295,7.090910,1.791759,7.313220,10.488743,9.812249,12.584560,9.186457,12.208056,9.131622,12.154042,10.525702,13.234633


In [28]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        #x = x.drop(['시간당총광량','일간누적총광량','일간누적총광량합'],axis=1)
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].std()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].std()
            day_light = day[day['일간누적총광량']>=0]['일간누적총광량'].std()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].std()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].std()

            day_white2 = day[day['일간누적백색광량']>=0]['일간누적백색광량'].mean()
            day_water2 = day[day['일간누적분무량']>=0]['일간누적분무량'].mean()
            day_light2 = day[day['일간누적총광량']>=0]['일간누적총광량'].mean()
            day_blue2 = day[day['일간누적청색광량']>=0]['일간누적청색광량'].mean()
            day_red2 = day[day['일간누적적색광량']>=0]['일간누적적색광량'].mean()

            time_list = day['obs_time'].unique()

            for col in col_list :
                if col in ['일간누적백색광량','일간누적청색광량','일간누적적색광량','일간누적분무량','일간누적총광량',
                            '일간누적분무량합', '일간누적백색광량합', '일간누적적색광량합', '일간누적청색광량합', '일간누적총광량합']:
                    for t in range(2,len(time_list),3):
                        time = time_list[t]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        x[col+str(time)] = value1*(i+1)
                        
                else :
                    for t in range(0,len(time_list)-2) :
                        time = time_list[t]
                        ntime = time_list[t+1]
                        nntime = time_list[t+2]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        value2 = day[day['obs_time']==ntime][col].iloc[0]
                        value3 = day[day['obs_time']==nntime][col].iloc[0]
                        x[col+str(time)+str("~")+str(nntime)] = (value1 + value2 + value3 )

            x['day_water_weight_std'] = (day_water)
            x['day_white_weight_std'] = (day_white)
            x['day_red_weight_std'] = (day_red)
            x['day_blue_weight_std'] = (day_blue)
            x['day_light_weight_std'] = (day_light)

            x['day_water_weight_mean'] = (day_water2*(i+1))
            x['day_white_weight_mean'] = (day_white2*(i+1))
            x['day_red_weight_mean'] = (day_red2*(i+1))
            x['day_blue_weight_mean'] = (day_blue2*(i+1))
            x['day_light_weight_mean'] = (day_light2*(i+1))

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [29]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data_re/상추/train_input/*.csv', 'drive/MyDrive/dacon/data_re/상추/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data_re/상추/test_input/*.csv', 'drive/MyDrive/dacon/data_re/상추/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [30]:
train.to_csv('drive/MyDrive/dacon/data_re/상추/train_log_1203.csv')
test.to_csv('drive/MyDrive/dacon/data_re/상추/test_log_1203.csv')

In [7]:
# train = pd.read_csv('drive/MyDrive/dacon/data_re/상추/train1203.csv').iloc[:,1:]
# test = pd.read_csv('drive/MyDrive/dacon/data_re/상추/test1203.csv').iloc[:,1:]

In [14]:
import sklearn.ensemble

In [ ]:
#  "random_state":42,
#       'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
#       'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#       "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
#       "max_depth":trial.suggest_int("max_depth", 4, 16),
#       'random_strength' :trial.suggest_int('random_strength', 0, 100),
#       "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
#       "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
#       "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#       "max_bin": trial.suggest_int("max_bin", 200, 500),
#       'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#   }

In [31]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost

X = train.drop(['predicted_weight_g'],axis=1)
y = train['predicted_weight_g']
CAT_model= {}
for f in range(0,28):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      CAT = catboost.CatBoostRegressor(ltask_type='GPU', n_estimators = , verbose=0,random_state=404)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')

====================================1============================================
0 27
1 Fold MAE = 13.409569467293576


====================================2============================================
28 55
2 Fold MAE = 8.143705115155276


====================================3============================================
56 83
3 Fold MAE = 29.401451328690705


====================================4============================================
84 111
4 Fold MAE = 26.29245740933074


====================================5============================================
112 139
5 Fold MAE = 30.477706764830764


====================================6============================================
140 167
6 Fold MAE = 17.44808503571146


====================================7============================================
168 195
7 Fold MAE = 13.521252841252586


====================================8============================================
196 223
8 Fold MAE = 8.40957339139874


=====================

In [32]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(28):
    pc += CAT_model[fold].predict(testX)/28
pc

array([  1.97962   ,   2.0820718 ,   2.66367179,   2.67882257,
         3.67574979,   3.15103058,   3.10297633,   3.60861819,
         4.37484973,   5.42149738,   8.3744445 ,  11.60693721,
        17.93975304,  22.65410638,  10.47447159,  27.27157981,
        33.31723683,  25.53030431,  22.71424133,  32.73572549,
        51.88225901,  57.79410742,  65.46031083,  72.50186593,
        80.52461193,  81.29226471,  86.66684663, 103.42858573,
         1.40071504,   0.97614555,   1.11586125,   1.06125416,
         1.3234767 ,   1.17746203,   2.0178752 ,   2.24307263,
         3.36088198,   4.62561873,   5.43670362,   8.67268666,
        10.99018765,  10.66768057,  16.86767988,  15.71541006,
        18.35077498,  23.1432121 ,  23.94995474,  30.98892018,
        45.03531368,  42.76130911,  43.77304689,  44.36181104,
        50.98304493,  45.10959545,  56.5947689 ,  62.7111329 ,
         1.81405022,   2.9133326 ,   3.10668237,   3.06582932,
         3.52083083,   4.27038813,   3.51084514,   3.97

In [34]:
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data_re/상추/test_target/*.csv'))
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1

In [35]:
corr_df = train.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
s = corr_df.unstack()
성장 = pd.DataFrame(s['predicted_weight_g']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
성장.sort_values(by=[0],ascending=False).reset_index().to_csv('target값이랑corr.csv',index=False,encoding='utf-8-sig')
under_1per = 성장[성장[0]<0.1].index
under_1per

Index(['시간당총광량9~11', '내부온도관측치19~21', '시간당백색광량20~22', '내부온도관측치2~4',
       '시간당백색광량21~23', 'day_red_weight_std', '내부온도관측치6~8', '내부온도관측치20~22',
       '시간당총광량8~10', 'day_water_weight_std', '시간당적색광량16~18',
       'day_blue_weight_std', '내부온도관측치21~23', '시간당적색광량14~16', '시간당적색광량15~17',
       '시간당적색광량13~15', '시간당백색광량0~2', '시간당적색광량17~19', '내부온도관측치5~7',
       '내부온도관측치3~5', '일간누적백색광량2', '일간누적백색광량5', '시간당총광량7~9', '시간당적색광량12~14',
       '시간당적색광량18~20', '내부온도관측치4~6', '시간당청색광량12~14', '시간당청색광량11~13',
       '시간당청색광량10~12', '시간당총광량6~8', '시간당분무량1~3', '시간당적색광량11~13', '시간당청색광량9~11',
       '시간당청색광량8~10', '시간당청색광량13~15', '시간당적색광량19~21', '시간당청색광량7~9',
       '시간당적색광량10~12', '시간당백색광량1~3', '시간당백색광량2~4', '시간당백색광량3~5'],
      dtype='object')

In [36]:
drop_column = train.copy()
drop_column = drop_column.drop(under_1per,axis=1)

In [39]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost

X = drop_column.drop(['predicted_weight_g'],axis=1)
y = drop_column['predicted_weight_g']
CAT_model= {}
for f in range(0,28):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      CAT = catboost.CatBoostRegressor(task_type='GPU', verbose=0,random_state=404)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')

====================================1============================================
0 27
1 Fold MAE = 12.124154706461573


====================================2============================================
28 55
2 Fold MAE = 8.326138198352682


====================================3============================================
56 83
3 Fold MAE = 28.66534742586578


====================================4============================================
84 111
4 Fold MAE = 24.571234951364776


====================================5============================================
112 139
5 Fold MAE = 30.530251041272226


====================================6============================================
140 167
6 Fold MAE = 15.811870347272999


====================================7============================================
168 195
7 Fold MAE = 13.529892735917894


====================================8============================================
196 223
8 Fold MAE = 8.225040722413489


===================

In [40]:
drop_columntt = test.copy()
drop_columntt = drop_columntt.drop(under_1per,axis=1)

In [41]:
testX = drop_columntt.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(28):
    pc += CAT_model[fold].predict(testX)/28
pc

array([  2.20181491,   2.20082895,   2.75938021,   3.12125817,
         4.01751984,   3.48599518,   3.26544529,   3.86986221,
         4.36800586,   5.75176172,   8.18236998,  11.29824812,
        17.03649024,  24.89762367,  10.87818537,  27.7155448 ,
        33.23161723,  27.57746698,  24.19075257,  33.53970404,
        52.51164623,  57.01494033,  63.75544542,  72.45447544,
        81.95964549,  79.16092111,  88.51791951, 104.79771834,
         1.587378  ,   1.16780071,   1.48758433,   1.35882949,
         1.6462992 ,   1.48686331,   2.16252784,   2.49458021,
         3.58654776,   5.09908755,   5.45030284,   8.07839607,
        10.22934892,  11.12671449,  17.30926271,  15.57899029,
        18.28821615,  22.48987116,  25.26403325,  28.88397188,
        41.89561218,  41.25851513,  40.82126849,  42.87506584,
        47.51917748,  43.6369322 ,  57.64651297,  63.19319387,
         2.18412526,   2.61757335,   2.92481055,   3.04384688,
         3.26336052,   3.74980593,   3.11790148,   3.64

In [42]:
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data_re/상추/test_target/*.csv'))
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1